In [ ]:
!sudo python start.py
import numpy as np
from useful import Robot, SpeechSynthesis
from XGO import XGOEDU
from xgolib import XGO
import time
import useful
import asyncio
import websockets
import threading
import os
from behavior import Behavior

# 初始化机器人
XGO_edu = XGOEDU()
g_car = XGO("xgorider")
robot = Robot(XGO_edu, g_car)
beh = Behavior(XGO_edu, g_car)

# 表情控制类
class EmotionController:
    def __init__(self, XGO_edu):
        self.XGO_edu = XGO_edu
        self.CN = True
        self.CN_cn = False
        self.EN = True
        self.EN_en = False
        self.Math = True
        self.Math_math = False
        self.Back = True
        self.Back_back = False
        self.lock = threading.Lock()

    def show_emo_CN(self):
        i = 0
        while self.CN:
            with self.lock:
                if self.CN_cn:
                    k = i % 87 + 1
                    self.XGO_edu.lcd_picture("speak{}.png".format(k), "RaspberryPi-CM4-main/demos/gptfree/speak4/")
                    i += 1
                else:
                    time.sleep(0.1)
            time.sleep(0.1)

    def show_emo_EN(self):
        i = 0
        while self.EN:
            with self.lock:
                if self.EN_en:
                    k = i % 10 + 1
                    self.XGO_edu.lcd_picture("{}.png".format(k), "RaspberryPi-CM4-main/demos/expression/awkwardness_r/")
                    i += 1
                else:
                    time.sleep(0.1)
            time.sleep(0.1)

    def show_emo_Math(self):
        i = 0
        while self.Math:
            with self.lock:
                if self.Math_math:
                    k = i % 8 + 1
                    self.XGO_edu.lcd_picture("{}.png".format(k), "RaspberryPi-CM4-main/demos/expression/naughty_r/")
                    i += 1
                else:
                    time.sleep(0.1)
            time.sleep(0.1)

    def show_emo_Back(self):
        i = 0
        while self.Back:
            with self.lock:
                if self.Back_back:
                    k = i % 8 + 1
                    self.XGO_edu.lcd_picture("{}.png".format(k), "RaspberryPi-CM4-main/demos/expression/cute_r/")
                    i += 1
                else:
                    time.sleep(0.1)
            time.sleep(0.1)

    def start_threads(self):
        thread_CN = threading.Thread(target=self.show_emo_CN)
        thread_CN.start()

        thread_EN = threading.Thread(target=self.show_emo_EN)
        thread_EN.start()

        thread_Math = threading.Thread(target=self.show_emo_Math)
        thread_Math.start()

        thread_Back = threading.Thread(target=self.show_emo_Back)
        thread_Back.start()

        return thread_CN, thread_EN, thread_Math, thread_Back


emotion_controller = EmotionController(XGO_edu)
thread_CN, thread_EN, thread_Math, thread_Back = emotion_controller.start_threads()
emotion_controller.CN_cn = True


def return_to_bk():
    emotion_controller.Back_back = False
    emotion_controller.Math_math = False
    emotion_controller.EN_en = False
    emotion_controller.CN_cn = True


# 状态抽象基类
class State:
    def __init__(self, context):
        self.context = context

    async def handle_message(self, websocket, message):
        # 默认处理无效指令
        await websocket.send("")

    async def enter(self, websocket):
        # 进入状态时的初始化操作
        pass


# 主页状态
class HomeState(State):
    async def handle_message(self, websocket, message):
        if message == "EN":
            self.context.transition_to(MainEnglishState(self.context))
            # g_car.rider_turn(30,1)
            emotion_controller.Back_back = True
            emotion_controller.Math_math = False
            emotion_controller.EN_en = False
            emotion_controller.CN_cn = False
            tran = "已进入趣味英语界面,我将为你打开一个新的英语世界"
            robot.speaking(tran)

        elif message == "CN":
            self.context.transition_to(TextbookLearningState(self.context))
            # g_car.rider_turn(30,1)
            emotion_controller.CN_cn = False
            emotion_controller.Back_back = False
            emotion_controller.Math_math = False
            emotion_controller.EN_en = True
            tran_1 = "已进入课本学习界面"
            robot.speaking(tran_1)
            tran_2 = "今天学习任务也很艰巨哦，学习第14页古诗山行,了解古诗释义,体验古诗情景,同时别忘了预习古诗赠刘景文,小方一直陪伴你学习左右"
            thread = threading.Thread(target=beh.action, args=('height', 6))
            thread.start()
            robot.speaking(tran_2)
            # await websocket.send(tran_1)
        elif message == "Math":
            self.context.transition_to(MathTeachingState(self.context))
            g_car.rider_turn(30, 1)
            emotion_controller.CN_cn = False
            emotion_controller.Back_back = False
            emotion_controller.Math_math = True
            emotion_controller.EN_en = False
            tran = "已进入数学讲解界面"
            robot.speaking(tran)
            await asyncio.sleep(3)
            # robot.action(0, 0.2, 0.5)
            robot.speaking("正在扫描中")
            await asyncio.sleep(1)
            robot.speaking("正在智能批改中")
            await asyncio.sleep(2)
            robot.speaking("批改完成了,点击按钮进行查看")
            # await websocket.send("批改完成")
        elif message == "report":
            self.context.transition_to(ReportState(self.context))
            g_car.rider_turn(30, 1)
            emotion_controller.CN_cn = False
            emotion_controller.Back_back = False
            emotion_controller.Math_math = False
            emotion_controller.EN_en = False
            XGO_edu.lcd_picture("report.png", "RaspberryPi-CM4-main/demos/gptfree/speak4/")
            # robot.speaking(tran)
            robot.speaking("请查看本周的学习报告")
            # robot.speaking("孩子本周学习表现优秀，数学解题能力显著提升，阅读量大幅增加，课堂参与度高，学习态度认真，继续保持，未来潜力无限！")
            # g_car.rider_turn(90,1)
            # await websocket.send("ok")
        elif message == "退出":
            # g_car.rider_turn(30,1)
            os.system("sudo mplayer " + "wang.wav" + " -novideo")
            return_to_bk()
            # await websocket.send("ding")
        else:
            await super().handle_message(websocket, message)


# 趣味英语状态
class MainEnglishState(State):
    def __init__(self, context):
        super().__init__(context)

    async def handle_message(self, websocket, message):
        if message == "photo":
            #self.context.first_photo = False
            g_car.rider_led(1,[int(np.random.rand()*100%256),int(np.random.rand()*100%256),int(np.random.rand()*100%256)])
            time.sleep(0.5)
            g_car.rider_led(1,[0,0,0])
            # g_car.rider_turn(30,1)
            robot.speaking("今天想听什么故事")
            if self.context.first_photo:
                await asyncio.sleep(3)
                print("是否为第一张", self.context.first_photo)
                robot.speaking("识别成功")
                await websocket.send("whale,toy,shark,seal,fish,octopus")
                robot.speaking("亲爱的小朋友,这是一只鲸鱼,真可爱，让我们一起来学习关于它的英语世界吧")
                self.context.first_photo = False

            else:
                await asyncio.sleep(3)
                print("是否为第一张", self.context.first_photo)
                robot.TakePhoto()
                robot.speaking("识别成功")
                res = robot.QwenAsk("/home/pi/xgoPictures/photo.jpg",
                                    "帮我识别我手上拿的物体，输出该物体的英语单词和同类词，每个单词以,间隔")
                speak = robot.QwenAsk("/home/pi/xgoPictures/photo.jpg",
                                      "帮我识别我手上拿的物体，以下面这种格式输出:亲爱的小朋友，这是，真可爱，让我们一起来学习它的英语世界吧")
                robot.speaking(speak)
                await websocket.send(res)
        elif message.startswith("read:"):
            txt = ''.join(message.split('.')[1:3])
            robot.speaking(txt)
            # await websocket.send("朗读完毕")
        elif "单词" in message:
            # g_car.rider_turn(30,1)
            robot.speaking("你选择了" + message + "现在为你生成一篇小短文")
            robot.speaking("仔细阅读这个有趣的小故事哦,希望你提升你的英语实力")
            # await websocket.send("生成完成")
        elif message == "退出":
            # g_car.rider_turn(30,1)
            self.context.transition_to(HomeState(self.context))
            return_to_bk()
            os.system("sudo mplayer " + "wang.wav" + " -novideo")
            # await websocket.send("ding")
        else:
            await super().handle_message(websocket, message)


# 课本学习状态
class TextbookLearningState(State):
    async def handle_message(self, websocket, message):
        if message == "read":
            # g_car.rider_turn(30,1)
            robot.speaking("正在为你阅读课文")
        elif "analysis" in str(message):
            message = str(message)
            res = robot.chat_with_gpt(message, "适当分析该课文解释，不超过100字，用中文")
            robot.speaking(res)
            # await websocket.send(res)
        elif "translation" in str(message):
            message = str(message)
            res = robot.chat_with_gpt(message, "这是课文翻译内容,适当用一句话概况，用中文")
            robot.speaking(res)
            # await websocket.send(res)
        elif "video:1" in message:
            robot.speaking("远上寒山石径斜，白云生处有人家。停车坐爱枫林晚，霜叶红于二月花。")
            # await websocket.send('ok')
        elif "video:2" in message:
            robot.speaking("荷尽已无擎雨盖，菊残犹有傲霜枝。一年好景君须记，最是橙黄橘绿时。")
            # await websocket.send('ok')
        elif message == "Xmind":
            robot.speaking("正在为你生成思维导图,有什么不会的及时问我哦")
        elif message == "退出":
            # g_car.rider_turn(30,1)
            self.context.transition_to(HomeState(self.context))
            return_to_bk()
            os.system("sudo mplayer " + "wang.wav" + " -novideo")
            # await websocket.send("ding")
        else:
            await super().handle_message(websocket, message)


# 数学讲解状态
class MathTeachingState(State):
    async def handle_message(self, websocket, message):
        if message == "teach":
            thread_r = threading.Thread(target=beh.action, args=('roll', 6))
            thread_r.start()
            robot.speaking("正在为你讲解错题,请看我们的虚拟黑板")
            thread_l = threading.Thread(target=beh.action, args=('light', 10))
            thread_l.start()
            # g_car.rider_turn(45,1)
            robot.speaking(
                "假设上个月我们有一个圈,那么这个月我们有四个圈,这样这个月比上个月就多三个圈了,多三个圈是多多少了?是21对不对,21除以3等于7,那么每一个圈代表7个,一共有1加4等于五份,所以最后的答案是35你明白了吗?")
            g_car.rider_periodic_z(2)
            await asyncio.sleep(2)
            g_car.rider_periodic_z(0)
            robot.speaking("讲解完成,还有什么不懂的吗?")
            recognized_text = robot.SpeechRecognition(robot)
            while useful.are_pronunciations_equal(recognized_text, "懂"):
                recognized_text = robot.SpeechRecognition(robot)
                txt = robot.chat_with_gpt(recognized_text,
                                          "你刚才为孩子解答了一道应用题目为某电器商店这个月的销售量是上个月的4倍，这个月的销售量又比上个月增加21台，问这两个月的销售量一共是多少台，但孩子还有疑问，解答孩子的问题,两句话以内")
                robot.speaking(txt)
                recognized_text = robot.SpeechRecognition(robot)
            g_car.rider_periodic_roll(2)
            await asyncio.sleep(3)
            g_car.rider_periodic_roll(0)
            # await websocket.send("已返回")
        elif message == "退出":
            # g_car.rider_turn(30,1)
            self.context.transition_to(HomeState(self.context))
            return_to_bk()
            os.system("sudo mplayer " + "wang.wav" + " -novideo")
            # await websocket.send("已返回主页")
        else:
            await super().handle_message(websocket, message)


# 学习报告状态
class ReportState(State):
    async def enter(self, websocket):
        robot.speaking("正在为你生成学习报告...")
        await asyncio.sleep(2)
        report = "本周学习报告：数学进步显著，阅读理解能力提升，建议加强英语口语练习。"
        robot.speaking(report)
        # await websocket.send(report)

    async def handle_message(self, websocket, message):
        if message == "退出":
            # g_car.rider_turn(30,1)
            self.context.transition_to(HomeState(self.context))
            return_to_bk()
            os.system("sudo mplayer " + "wang.wav" + " -novideo")
            # await websocket.send("ding")
        else:
            await super().handle_message(websocket, message)


# 上下文类
# 修改RobotContext类
class RobotContext:
    def __init__(self):
        self._state = HomeState(self)
        self.last_message = None  # 只添加这一行来记录上一条消息
        self.first_photo = True

    def transition_to(self, state):
        self._state = state

    async def handle_message(self, websocket, message):
        # 检查是否与上一条消息相同
        if message == self.last_message:
            print(f"忽略重复消息: {message}")
            return  # 直接返回，不执行任何操作

        # 更新上一条消息记录
        self.last_message = message

        # 正常处理消息
        await self._state.handle_message(websocket, message)


import asyncio
import websockets
from functools import partial

# 存储不同端口的客户端连接
clients = {
    6790: set(),
    6791: set()
}


async def safe_send(client, message):
    """安全发送消息"""
    try:
        await client.send(message)
    except websockets.ConnectionClosed:
        print("连接已关闭，无法发送消息")


async def handler(websocket, path, port):
    """处理WebSocket连接"""
    clients[port].add(websocket)
    print(f"客户端连接至端口 {port}，当前连接数：{len(clients[port])}")

    try:
        async for message in websocket:
            print(f"[端口{port}] 收到原始消息: {message}")

            # 调用RobotContext的handle_message方法处理消息
            processed_message = await context.handle_message(websocket, message)

            # 双向转发逻辑
            if "post" in message.lower() and not message.startswith("[转发]"):
                target_port = 6791 if port == 6790 else 6790
                forwarded_msg = message

                print(f"准备转发到端口 {target_port}")
                if clients[target_port]:
                    await asyncio.gather(
                        *[safe_send(c, forwarded_msg) for c in clients[target_port]],
                        return_exceptions=True
                    )
    except websockets.ConnectionClosed:
        print(f"端口 {port} 连接断开")
    finally:
        clients[port].remove(websocket)
        print(f"端口 {port} 剩余连接：{len(clients[port])}")


async def main():
    """启动双端口服务"""
    global context
    context = RobotContext()
    server_6790 = await websockets.serve(
        partial(handler, port=6790),
        "0.0.0.0", 6790
    )
    server_6791 = await websockets.serve(
        partial(handler, port=6791),
        "0.0.0.0", 6791
    )
    print("双端口服务已启动：6790 & 6791")
    await asyncio.Future()


# 在Jupyter Notebook中使用await运行异步函数
await main()

杀死进程号: 943
杀死进程号: 944
未找到运行 python3 main.py 的进程
分辨率: 1280.0x720.0
亮度: 65.0
对比度: 10.0
摄像头已成功设置
Init Done.
双端口服务已启动：6790 & 6791
客户端连接至端口 6791，当前连接数：1
[端口6791] 收到原始消息: Math


ALSA lib pcm.c:2660:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.front
ALSA lib pcm.c:2660:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.rear
ALSA lib pcm.c:2660:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.center_lfe
ALSA lib pcm.c:2660:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.side
ALSA lib pcm.c:2660:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.surround21
ALSA lib pcm.c:2660:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.surround21
ALSA lib pcm.c:2660:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.surround40
ALSA lib pcm.c:2660:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.surround41
ALSA lib pcm.c:2660:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.surround50
ALSA lib pcm.c:2660:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.surround51
ALSA lib pcm.c:2660:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.surround71
ALSA lib pcm.c:2660:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.iec958
ALSA lib pcm.c:2660:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.iec958
ALSA lib pcm.c:2660:(snd

音频播放完成
已进入数学讲解界面


ALSA lib pcm.c:2660:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.front
ALSA lib pcm.c:2660:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.rear
ALSA lib pcm.c:2660:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.center_lfe
ALSA lib pcm.c:2660:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.side
ALSA lib pcm.c:2660:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.surround21
ALSA lib pcm.c:2660:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.surround21
ALSA lib pcm.c:2660:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.surround40
ALSA lib pcm.c:2660:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.surround41
ALSA lib pcm.c:2660:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.surround50
ALSA lib pcm.c:2660:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.surround51
ALSA lib pcm.c:2660:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.surround71
ALSA lib pcm.c:2660:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.iec958
ALSA lib pcm.c:2660:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.iec958
ALSA lib pcm.c:2660:(snd

音频播放完成
正在扫描中


...

do_connect: could not connect to socket
connect: No such file or directory
Failed to open LIRC support. You will not be able to use your remote control.
AO: [pulse] Init failed: Connection refused
Failed to initialize audio driver 'pulse'


AO: [alsa] 48000Hz 2ch s16le (2 bytes per sample)
Video: no video
Starting playback...
A:   0.3 (00.2) of 0.5 (00.5)  0.7% 


Exiting... (End of file)
[端口6791] 收到原始消息: CN


ALSA lib pcm.c:2660:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.front
ALSA lib pcm.c:2660:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.rear
ALSA lib pcm.c:2660:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.center_lfe
ALSA lib pcm.c:2660:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.side
ALSA lib pcm.c:2660:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.surround21
ALSA lib pcm.c:2660:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.surround21
ALSA lib pcm.c:2660:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.surround40
ALSA lib pcm.c:2660:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.surround41
ALSA lib pcm.c:2660:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.surround50
ALSA lib pcm.c:2660:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.surround51
ALSA lib pcm.c:2660:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.surround71
ALSA lib pcm.c:2660:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.iec958
ALSA lib pcm.c:2660:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.iec958
ALSA lib pcm.c:2660:(snd

音频播放完成
已进入课本学习界面
当前线程号为：Thread-12


ALSA lib pcm.c:2660:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.front
ALSA lib pcm.c:2660:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.rear
ALSA lib pcm.c:2660:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.center_lfe
ALSA lib pcm.c:2660:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.side
ALSA lib pcm.c:2660:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.surround21
ALSA lib pcm.c:2660:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.surround21
ALSA lib pcm.c:2660:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.surround40
ALSA lib pcm.c:2660:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.surround41
ALSA lib pcm.c:2660:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.surround50
ALSA lib pcm.c:2660:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.surround51
ALSA lib pcm.c:2660:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.surround71
ALSA lib pcm.c:2660:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.iec958
ALSA lib pcm.c:2660:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.iec958
ALSA lib pcm.c:2660:(snd